# Assignment Customer Satisfaction and Sentiment Analysis


## Objective

You are a data analyst of a consulting company that provides customer insight regarding multiple ticketing system, such as JIRA and Zoho Desk. Your team gather surveys to customers regarding their ticketing system's performance. Your role in the team is to gather reports regarding customer satisfaction and sentiment analysis into a single dashboard and present your insight.

Analyze the following metrics and other insight you can find in the dataset:

- Survey response rate
- Customer Satisfaction score (CSAT)
- Customer Effort Score (CES)
- Net Promoter Score (NPS)
- Sentiment Analysis



## Data Preparation

In [1]:
import numpy as np
import pandas as pd
import os

pd.options.display.max_columns = 999
pd.options.display.float_format = "{:.2f}".format

### Import Dataset

In [2]:
df_ticket = pd.read_csv('/content/assignment_ticket_system_review.csv')

### Read Data

Read the file **assignment_ticket_system_review.csv**

In [3]:
# Read Data
df_ticket.head()

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text
0,T_02161,2024-11-20,Zendesk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T_00229,2024-10-06,Zendesk,3.00,4.00,3.00,3.00,2.00,6.00,Customer tickets managements
2,T_04527,2024-12-26,Zoho Desk,5.00,5.00,5.00,5.00,5.00,8.00,"After 6 months of using the Zoho desk, we shif..."
3,T_03190,2024-12-08,Zoho Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T_00644,2024-10-17,Zendesk,5.00,3.00,4.00,5.00,5.00,6.00,Pros:Zendesk has always been one of the go-to ...


The following is the dictionary for the data, survey is only valid if all of the survey questions and text review is not blank (null):

**General Information**
- id_survey: identifier for each survey
- date_of_survey: date of survey taken
- ticket_system: The name of the ticket system being reviewed (e.g. Zoho Desk)

**Survey Questions**
- overall_rating: The overall satisfaction rating given by the reviewer, ranging from 1 to 5
- customer_service: The satisfaction rating for the customer service provided by the ticket system, ranging from 1 to 5.
- features: The satisfaction rating for the features of the ticket system, ranging from 1 to 5
- value_for_money: The satisfaction rating for the value for money provided by the ticket system, ranging from 1 to 5
- ease_of_use: The rating for how easy the ticket system is to use, ranging from 1 to 5
- likelihood_to_recommend: The likelihood that the reviewer would recommend the ticket system to others, ranging from 1 to 10
- overall_text: The full text of the overall review, providing detailed feedback on the ticket system.


In [4]:
# Check the type of data
df_ticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_survey                1462 non-null   object 
 1   date_of_survey           1462 non-null   object 
 2   ticket_system            1462 non-null   object 
 3   overall_rating           787 non-null    float64
 4   customer_service         787 non-null    float64
 5   features                 787 non-null    float64
 6   value_for_money          787 non-null    float64
 7   ease_of_use              787 non-null    float64
 8   likelihood_to_recommend  787 non-null    float64
 9   overall_text             787 non-null    object 
dtypes: float64(6), object(4)
memory usage: 114.3+ KB


### Data Cleansing

Convert the date column into proper date_time format.

In [5]:
# Convert data type
df_ticket['date_of_survey'] = pd.to_datetime(df_ticket['date_of_survey'])
df_ticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id_survey                1462 non-null   object        
 1   date_of_survey           1462 non-null   datetime64[ns]
 2   ticket_system            1462 non-null   object        
 3   overall_rating           787 non-null    float64       
 4   customer_service         787 non-null    float64       
 5   features                 787 non-null    float64       
 6   value_for_money          787 non-null    float64       
 7   ease_of_use              787 non-null    float64       
 8   likelihood_to_recommend  787 non-null    float64       
 9   overall_text             787 non-null    object        
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 114.3+ KB


In [6]:
df_ticket['date_of_survey'].sort_values(ascending=False).head(10)

,date_of_survey
1432,2024-12-30
1305,2024-12-30
1445,2024-12-30
269,2024-12-30
1083,2024-12-30
910,2024-12-30
1256,2024-12-30
399,2024-12-30
887,2024-12-30
1422,2024-12-30


## Survey Analysis

### Response Rate

Start by analyzing how many customers has filled the survey, indicated by whether the overall_rating is not blank.

In [7]:
# How many customer responded to the survey?
df_ticket['fill_survey'] = np.where(df_ticket['overall_rating'].isnull(), 'Not Responded', 'Responded')
df_ticket.value_counts('fill_survey', normalize = True).reset_index()

,fill_survey,proportion
0,Responded,0.54
1,Not Responded,0.46


Create a new dataframe that consists only of those who have responded the survey to simplify calculating the CSAT, CES, and NPS Score.

In [8]:
# Responded Customer
df_responded = df_ticket[ df_ticket['fill_survey'] == 'Responded'].copy()
df_responded.head()

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text,fill_survey
1,T_00229,2024-10-06,Zendesk,3.00,4.00,3.00,3.00,2.00,6.00,Customer tickets managements,Responded
2,T_04527,2024-12-26,Zoho Desk,5.00,5.00,5.00,5.00,5.00,8.00,"After 6 months of using the Zoho desk, we shif...",Responded
4,T_00644,2024-10-17,Zendesk,5.00,3.00,4.00,5.00,5.00,6.00,Pros:Zendesk has always been one of the go-to ...,Responded
6,T_04682,2024-12-28,Zoho Desk,5.00,4.00,5.00,5.00,5.00,8.00,It has been very useful so far to integrate mu...,Responded
8,T_01238,2024-11-02,Freshdesk,4.00,4.00,4.00,5.00,4.00,8.00,Pros:It's easy to use and very intuitive.We ha...,Responded


### CSAT Score

Measure the customer's overall satisfaction score (CSAT) with the following formula:

$$
CSAT = \frac{\Sigma\ total\ satisfaction\ score}{number\ of\ responded\ customer \times \max\ rating}
$$

The max rating is inserted to convert the CSAT score into percentage.

CSAT score can be classified into categories based on the result. There is no absolute threshold for each categories but the following is the common threshold:

- \>= 90%: Excellent
- 75%-90%: Good
- 60-75%: Fair
- \<60%: Poor

In [9]:
# CSAT Score
max_rating = 5
n_data = (df_responded.shape[0] * max_rating)

csat_score = df_responded['overall_rating'].sum() / n_data

print(f'Overall CSAT Score: { (csat_score * 100):.1f}%')

Overall CSAT Score: 91.2%


Measure the satisfaction score for the following attributes:

- customer service
- features
- value for money

In [10]:
# Satisfaction Score for Attributes
score_customer_service = df_responded['customer_service'].sum() / n_data
score_features = df_responded['features'].sum()/ n_data
score_value_for_money = df_responded['value_for_money'].sum()/ n_data

print(f'Overall CSAT Score: { (csat_score * 100):.1f}%')
print(f'Customer Services: { (score_customer_service * 100):.1f}%')
print(f'Features: { (score_features * 100):.1f}%')
print(f'Value for Money: { (score_value_for_money * 100):.1f}%')

Overall CSAT Score: 91.2%
Customer Services: 67.3%
Features: 88.3%
Value for Money: 87.6%


### CES Score

Measure CES with the following formula


$$
CES = \frac{\Sigma\ total\ effort\ score}{number\ of\ responded\ customer \times \max\ rating}
$$

In [11]:
# CES Score
max_rating = 5

ces_ease_of_use = df_responded['ease_of_use'].sum()/ n_data

print(f'Ease of Use CES Score: { (ces_ease_of_use * 100):.1f}%')

Ease of Use CES Score: 89.5%


### NPS Score

To calculate the NPS score, first we must convert the **would_you_recommend** column into proper NPS Category based on the rating value:

* Promoter: Rating 9-10
* Passive: Rating 7-8
* Detractor: Rating < 7

In [12]:
#Mengecek kolom nun dan unique
df_responded['likelihood_to_recommend'].isna().sum()
df_responded['likelihood_to_recommend'].unique()

array([ 6.,  8.,  7.,  5., 10.,  9.,  3.,  4.,  2.,  1.,  0., -1.])

In [13]:
#Menampilkan yang nilanya tidak integer
df_responded[df_responded['likelihood_to_recommend'] == -1]

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text,fill_survey
938,T_04185,2024-12-21,Freshdesk,2.00,0.00,4.00,2.00,4.00,-1.00,Pros:The platform is relatively user friendly ...,Responded


In [14]:
# Hapus data dengan nilai tidak valid (misal -1)
df_responded = df_responded[df_responded['likelihood_to_recommend'].between(0, 10)]

In [15]:
# Buat kondisi dan nilai
nps_condition = [
    df_responded['likelihood_to_recommend'] >= 9,
    (df_responded['likelihood_to_recommend'] >= 7) & (df_responded['likelihood_to_recommend'] < 9),
    (df_responded['likelihood_to_recommend'] < 7)
]

nps_value = ['Promoter', 'Passive', 'Detractor']

# Tambahkan default agar semua hasil tetap string
df_responded['nps_category'] = np.select(nps_condition, nps_value, default='')

# Cek distribusi hasil
df_responded['nps_category'].value_counts(normalize=True)

,proportion
nps_category,
Passive,0.48
Promoter,0.32
Detractor,0.20



Calculate the NPS Score with the following formula

$$
NPS = \frac{Promoter - Detractor}{Total\ Survey\ Responded}
$$

In [16]:
# NPS Score
nps_agg_1 = df_responded.value_counts('nps_category').reset_index()

nps_agg_promoter = nps_agg_1[ nps_agg_1['nps_category'] == 'Promoter' ]['count'].item()
nps_agg_detractor = nps_agg_1[ nps_agg_1['nps_category'] == 'Detractor' ]['count'].item()
nps_score = (nps_agg_promoter - nps_agg_detractor)/nps_agg_1['count'].sum()

print(f'NPS Score: { (nps_score * 100):.1f}')

NPS Score: 12.1


NPS score can be ranging from -100 (when all customers are detractor) to 100 (when all customers are promoter).

NPS Score can be classified into categories based on the following threshold:

- \>= 70: Excellent
- 50-69: Very Good
- 30-49: Good
- 0-29: Average
- \< 0: Poor

In [17]:
df_ticket = df_ticket.merge(df_responded[['id_survey', 'nps_category']], on = 'id_survey', how = 'left')

df_ticket.head()

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text,fill_survey,nps_category
0,T_02161,2024-11-20,Zendesk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Responded,NaN
1,T_00229,2024-10-06,Zendesk,3.00,4.00,3.00,3.00,2.00,6.00,Customer tickets managements,Responded,Detractor
2,T_04527,2024-12-26,Zoho Desk,5.00,5.00,5.00,5.00,5.00,8.00,"After 6 months of using the Zoho desk, we shif...",Responded,Passive
3,T_03190,2024-12-08,Zoho Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Responded,NaN
4,T_00644,2024-10-17,Zendesk,5.00,3.00,4.00,5.00,5.00,6.00,Pros:Zendesk has always been one of the go-to ...,Responded,Detractor


## Sentiment Analysis

Create a new dataframe with no blank overall_text.

In [18]:
# Create new dataframe
df_ticket_clean = df_ticket.dropna(axis =0, subset = 'overall_text')[['id_survey', 'overall_text']].copy()

df_ticket_clean.head()

,id_survey,overall_text
1,T_00229,Customer tickets managements
2,T_04527,"After 6 months of using the Zoho desk, we shif..."
4,T_00644,Pros:Zendesk has always been one of the go-to ...
6,T_04682,It has been very useful so far to integrate mu...
8,T_01238,Pros:It's easy to use and very intuitive.We ha...


### Text Cleansing

In order to get more accurate sentiment, several text cleansing need to be done. However, in most of recent sentiment analysis models and algorithm, the only text cleansing needed are as follows:

* Clean double whitespace
* Clean URL/website
* Clean username (mostly in social media or digital text)

In [19]:
import re

def cleansing_text(x):
  # clean double whitespace
  out_text = ' '.join(x.split())

  # clean url
  out_text = re.sub(r"http\S+|www\S+|https\S+", 'http', out_text)

  # clean username
  out_text = re.sub(r"@\S+", '@user', out_text)

  return(out_text)

cleansing_text(" Doesn't  dissapoint. The car       was great. It was the best car rental experiences I've had! Salute to @jone who recommend https:/rental.com")

"Doesn't dissapoint. The car was great. It was the best car rental experiences I've had! Salute to @user who recommend http"

In [20]:
# apply cleansing to review
df_ticket_clean['clean_text'] = df_ticket_clean['overall_text'].apply(cleansing_text)

df_ticket_clean.tail()

,id_survey,overall_text,clean_text
1457,T_01536,"IT was not prioritizing, tracking, resolving a...","IT was not prioritizing, tracking, resolving a..."
1458,T_00582,"Very useful, one-stop solution with related to...","Very useful, one-stop solution with related to..."
1459,T_01834,We have done quite well. We have already more ...,We have done quite well. We have already more ...
1460,T_03629,"Immensely positive, highly recommended custome...","Immensely positive, highly recommended custome..."
1461,T_01981,Pros:Easy to start with a basic free setup and...,Pros:Easy to start with a basic free setup and...


### Sentiment Analysis

Create a sentiment categories using algorithm of your own choice.

In [21]:
# Sentiment Algorithm
%%capture
!pip install transformers
from transformers import pipeline

# sentiment analysis task with twitter roberta model
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

In [22]:
%%time
# Predict sentiment
transformer_results = sentiment_pipeline(df_ticket_clean['clean_text'].tolist())

df_ticket_clean['sentiment_score'] = transformer_results
df_ticket_clean['sentiment'] = df_ticket_clean['sentiment_score'].apply(lambda x: x['label'])

# Create sentiment category
sent_value = ['Negative', 'Neutral', 'Positive']
sent_condition = [df_ticket_clean['sentiment'] == 'LABEL_0',
                  df_ticket_clean['sentiment'] == 'LABEL_1',
                  df_ticket_clean['sentiment'] == 'LABEL_2'
                  ]

df_ticket_clean['sentiment'] = np.select(sent_condition, sent_value, default='Unknown')

df_ticket_clean.head()

CPU times: user 10.2 s, sys: 658 ms, total: 10.8 s
Wall time: 15.6 s


,id_survey,overall_text,clean_text,sentiment_score,sentiment
1,T_00229,Customer tickets managements,Customer tickets managements,"{'label': 'LABEL_1', 'score': 0.732880175113678}",Neutral
2,T_04527,"After 6 months of using the Zoho desk, we shif...","After 6 months of using the Zoho desk, we shif...","{'label': 'LABEL_2', 'score': 0.9796792268753052}",Positive
4,T_00644,Pros:Zendesk has always been one of the go-to ...,Pros:Zendesk has always been one of the go-to ...,"{'label': 'LABEL_2', 'score': 0.7879296541213989}",Positive
6,T_04682,It has been very useful so far to integrate mu...,It has been very useful so far to integrate mu...,"{'label': 'LABEL_2', 'score': 0.9632710218429565}",Positive
8,T_01238,Pros:It's easy to use and very intuitive.We ha...,Pros:It's easy to use and very intuitive.We ha...,"{'label': 'LABEL_2', 'score': 0.8690707683563232}",Positive


Check the number of data by sentiment.

In [23]:
# Number of Sentiment
df_ticket_clean['sentiment'].value_counts()

,count
sentiment,
Positive,667
Neutral,100
Negative,20


## Finalize Data for Reporting

Save the review data with NPS category and sentiment information to new csv for the dashboard.

In [24]:
# Save Data
df_ticket_final = df_ticket.merge(df_ticket_clean[['id_survey', 'sentiment']],
                                  on = 'id_survey', how = 'left'
                                  )

df_ticket_final.head()

,id_survey,date_of_survey,ticket_system,overall_rating,customer_service,features,value_for_money,ease_of_use,likelihood_to_recommend,overall_text,fill_survey,nps_category,sentiment
0,T_02161,2024-11-20,Zendesk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Responded,NaN,NaN
1,T_00229,2024-10-06,Zendesk,3.00,4.00,3.00,3.00,2.00,6.00,Customer tickets managements,Responded,Detractor,Neutral
2,T_04527,2024-12-26,Zoho Desk,5.00,5.00,5.00,5.00,5.00,8.00,"After 6 months of using the Zoho desk, we shif...",Responded,Passive,Positive
3,T_03190,2024-12-08,Zoho Desk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Responded,NaN,NaN
4,T_00644,2024-10-17,Zendesk,5.00,3.00,4.00,5.00,5.00,6.00,Pros:Zendesk has always been one of the go-to ...,Responded,Detractor,Positive


In [25]:
# Save data to excel
df_ticket_final.to_excel("ticket_final.xlsx", index=False)